In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from zipfile import ZipFile
import pandas

In [19]:
alldats = []
taxidir = Path('taxidata')
for n in range(1, 13):
    path = taxidir / ('trip_data_%d.csv.zip' % n)
    with path.open('rb') as fd:
        unzipped = ZipFile(fd)
        fname, = unzipped.namelist()
        
        with unzipped.open(fname) as fd:
            for chunk_no, chunk in enumerate(pandas.read_csv(fd, chunksize=100000, parse_dates=[5,6]),1): #dtype=dtypes), 1):
                ix = chunk.pickup_datetime < datetime.datetime(2013,1,2)
                ix = ix & (chunk.pickup_latitude > 40) & (chunk.pickup_latitude < 42)
                ix = ix & (chunk.pickup_longitude < -73) & (chunk.pickup_longitude > -75)
                n = sum(ix)
                if n > 0:
                    alldats.append(chunk[ix])
                
                if n > 0 or chunk_no % 10 == 0:
                    print(chunk_no, n)
            print('Data chunks:', len(alldats))
    break

1 22
2 45
3 75
4 32
5 71
6 57
7 74
8 75
9 46
10 90824
11 97673
12 16840
13 92
14 78
15 37
16 66
17 74
18 91
19 199
20 62
21 23
24 627
25 64558
26 8481
27 14
30 0
40 0
46 18307
47 25471
48 4067
50 0
60 0
61 528
62 8765
63 475
67 518
68 3540
69 8509
70 12721
71 15341
72 3041
80 0
90 0
100 0
110 0
117 3150
118 723
119 684
120 0
121 3
122 79
123 45
124 40
125 73
126 45
127 69
128 57
129 55
130 65
131 47
132 65
133 60
134 114
135 145
136 3796
137 4059
140 0
146 460
147 5799
148 4132
Data chunks: 60


In [12]:
concatted = pandas.concat(alldats)
tosave = concatted.iloc[:, 7:]

In [15]:
save_path.stem

'firstday.csv'

In [18]:
save_path = taxidir / 'firstday.csv.zip'
with ZipFile(str(save_path), mode='w') as zipped:
    csv = tosave.to_csv(index=False)
    zipped.writestr(save_path.stem, csv)

In [21]:
class ReadTell:
    def __init__(self, fd):
        self._fd = fd
        self._eof = False
        self._tell = 0
    
    def _update_tell(self, out):
        if len(out) == 0:
            self._eof = True
        else:
            self._tell += len(out)
        return out
    
    def read(self, *args):
        return self._update_tell(self._fd.read(*args))
    
    def readline(self, *args):
        return self._update_tell(self._fd.readline(*args))
    
    def readlines(self, *args):
        return self._update_tell(self._fd.readlines(*args))
    
    def close(self):
        self._fd.close()
        self._eof = True
    
    def done(self):
        return self._eof
    
    def tell(self):
        return self._tell
    
    def __enter__(self):
        return self
    
    def __exit__(self,*args):
        self.close()

In [36]:
path = taxidir / ('trip_data_1.csv.zip')
with path.open('rb') as fd:
    unzipped = ZipFile(fd)
    fname, = unzipped.namelist()
    with unzipped.open(fname) as fd:
        for n,chunk in enumerate(pandas.read_csv(fd, chunksize=10)):
            print(chunk)
            if n >= 3: break

                          medallion                      hack_license  \
0  89D227B655E5C82AECF13C3F540D4CF4  BA96DE419E711691B9445D6A6307C170   
1  0BD7C8F5BA12B88E0B67BED28BEA73D8  9FD8F69F0804BDB5549F40E9DA1BE472   
2  0BD7C8F5BA12B88E0B67BED28BEA73D8  9FD8F69F0804BDB5549F40E9DA1BE472   
3  DFD2202EE08F7A8DC9A57B02ACB81FE2  51EE87E3205C985EF8431D850C786310   
4  DFD2202EE08F7A8DC9A57B02ACB81FE2  51EE87E3205C985EF8431D850C786310   
5  20D9ECB2CA0767CF7A01564DF2844A3E  598CCE5B9C1918568DEE71F43CF26CD2   
6  496644932DF3932605C22C7926FF0FE0  513189AD756FF14FE670D10B92FAF04C   
7  0B57B9633A2FECD3D3B1944AFC7471CF  CCD4367B417ED6634D986F573A552A62   
8  2C0E91FF20A856C891483ED63589F982  1DA2F6543A62B8ED934771661A9D2FA0   
9  2D4B95E2FA7B2E85118EC5CA4570FA58  CD2F522EEE1FF5F5A8D8B679E23576B3   

  vendor_id  rate_code store_and_fwd_flag      pickup_datetime  \
0       CMT          1                  N  2013-01-01 15:11:48   
1       CMT          1                  N  2013-01-06 00:18:35  

In [42]:
pandas.read_csv?
dtype={'medallion':np.str}

Index(['medallion', 'hack_license', 'vendor_id', 'rate_code',
       'store_and_fwd_flag', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'trip_time_in_secs', 'trip_distance',
       'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude'],
      dtype='object')

In [38]:
dtypes={'pickup_datetime': np.datetime64, 'dropoff_datetime':  np.datetime64}

In [23]:
alldats = []
taxidir = Path('taxidata')
for n in range(1, 13):
    path = taxidir / ('trip_data_%d.csv.zip' % n)
    with path.open('rb') as fd:
        unzipped = ZipFile(fd)
        fname, = unzipped.namelist()
        
        headernames = None
        data_chunks = 0
        last_tell = 0
        with ReadTell(unzipped.open(fname)) as fd:
            while not fd.done():
                if headernames is None:
                    data = pandas.read_csv(fd, nrows=100000)
                    headernames = data.columns
                else:
                    data = pandas.read_csv(fd, nrows=100000, header=None, names=headernames)
                ix = pandas.to_datetime(data.pickup_datetime, errors='coerce') < datetime.datetime(2013,1,2)
                n = sum(ix)
                data_chunks += 1
                if n > 0:
                    alldats.append(data[ix])
                
                if n > 0 or data_chunks % 10 == 0:
                    print(data_chunks, n)
            print('Data chunks:', data_chunks)
    break

1 29
2 51
3 79
4 46
5 62
6 60
7 77
8 96
9 1900
10 99233
11 96163
12 8858
13 78
14 79
15 41
16 74
17 79
18 100
19 207
20 63
21 5
24 8812
25 61948
26 3850
27 3
46 33876
47 13281
48 1426
61 6626
62 3232
63 42
66 94
67 2690
68 1349
69 20875
70 1274
71 18069
72 14
101 1
109 3
112 1
113 1
116 3203
117 459
118 954
120 3
121 83
122 48
123 42
124 76
125 46
126 71
127 58
128 63
129 60
130 44
131 84
132 52
133 120
134 682
135 3669
136 3715
145 1380
146 5900
147 3228
Data chunks: 147


/usr/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
len(alldats)

In [ ]:
pandas.to_datetime(concatted.pickup_datetime).max()

In [ ]:
speeds = data.trip_distance / (data.trip_time_in_secs / 3600)

In [ ]:
speeds_sported = speeds[np.isfinite(speeds) & (speeds < 120)].sort_values()
speeds_sported[-10:]

In [ ]:
hist?

In [ ]:
speeds_sported.hist(bins=ceil(sqrt(len(speeds_sported))), histtype='step', linewidth=3, color=green)

In [ ]:
speeds.hist()

In [ ]:
taxidir = Path('taxidata')
for n in range(1, 13):
    path = taxidir / ('trip_data_%d.csv.zip' % n)
    with path.open('rb') as fd:
        unzipped = ZipFile(fd)
        fname, = unzipped.namelist()
        for lineno, line in enumerate(unzipped.open(fname), 1):
            print(fname, '::', lineno, '::', line.decode('utf-8'))
            if lineno >= 20:
                break
    break

In [ ]:
for n in range(1, 13):
    trip_url = 'https://nyctaxitrips.blob.core.windows.net/data/trip_data_%d.csv.zip' % n
    path = taxidir / ('trip_data_%d.csv.zip' % n)
    if path.exists(): continue
    print(path)
    request = requests.get(trip_url, stream=True)
    break

In [ ]:
length = int(request.headers['Content-Length'])

In [ ]:
for n in range(1, 10):
    if n % 5 == 0:
        print('Found', n)
        break
else:
    print('Not Found')

In [ ]:
import math

In [ ]:
math.ceil(894.2)